#### Importing Modules

In [44]:
from dataclasses import dataclass
from typing import Optional
from enum import Enum
import pandas as pd

#### Reading the Data

In [45]:
class InfillPattern(Enum):
    GRID = "grid"
    HONEYCOMB = "honeycomb"

In [46]:
class Material(Enum):
    ABS = "abs"
    PLA = "pla"

In [80]:
@dataclass
class ThreeDPrinterData:
    layer_height: float
    wall_thickness: int
    infill_density: int
    infill_pattern: InfillPattern
    nozzle_temperature: int
    bed_temperature: int
    print_speed: int
    material: Material
    fan_speed: int
    roughness: int
    tensile_strength: int
    elongation: float
    
    def __post_init__(self):
        # perform any validations on the input data
        pass
            
    
    def as_dict(self):
        return {
            'layer_height': self.layer_height,
            'wall_thickness': self.wall_thickness,
            'infill_density': self.infill_density,
            'infill_pattern': self.infill_pattern.value,
            'nozzle_temperature': self.nozzle_temperature,
            'bed_temperature': self.bed_temperature,
            'print_speed': self.print_speed,
            'material': self.material.value,
            'fan_speed': self.fan_speed,
            'roughness': self.roughness,
            'tensile_strength': self.tensile_strength,
            'elongation': self.elongation,
        }

In [81]:
df1 = ThreeDPrinterData(0.02, 8, 90, InfillPattern.GRID, 220, 60, 40, Material.ABS, 0, 25, 18, 1.2)

In [79]:
print(df1.as_dict())

{'layer_height': 0.02, 'wall_thickness': 8, 'infill_density': 90, 'infill_pattern': 'grid', 'nozzle_temperature': 220, 'bed_temperature': 60, 'print_speed': 40, 'material': 'abs', 'fan_speed': 0, 'roughness': 25, 'tensile_strength': 18, 'elongation': 1.2}
